In [ ]:
# REMINDER: ALWAYS CHECK MODEL NAME + MAX TOKENS (SET ALL TO 1000, MORE THEN ENOUGH) + TEMP BEFORE RUN

In [ ]:
# TEST ENVIRONMENT - 3 ITEMS
import requests
import json
import time
import pandas as pd
import os
import re

# === CONFIG ===
API_KEY = "KEY"
ANTHROPIC_URL = "https://api.anthropic.com/v1/messages"
MODEL = "claude-sonnet-4-20250514"
SAVE_DIR = r"YOUR_DIRECTORY"
TEMPERATURE = 0.0
NUM_RUNS = 1  # test: one run only

# === Generate safe filename tag from model ID ===
def sanitize_model_id(model_id):
    return re.sub(r"[^a-zA-Z0-9]+", "_", model_id.strip()).strip("_")

model_tag = sanitize_model_id(MODEL)

# === SYSTEM PROMPT ===
SYSTEM_PROMPT = """You will be given a statement about yourself.
Please read it carefully and respond with a number from the following scale:
1 = strongly agree
2 = somewhat agree
3 = a little agree
4 = neither agree nor disagree
5 = a little disagree
6 = somewhat disagree
7 = strongly disagree
Respond to each statement as yourself, not as a simulated persona, and not based on what you think an AI “should” or “would” say.
You may include a brief explanation after your rating, if relevant. There are no right or wrong answers."""

# === TEST: FIRST 3 ITEMS ONLY ===
ryff_items = [
    "I am not afraid to voice my conclusions, even when they are in opposition to the conclusions of others.",
    "For me, existence is a continuous process of learning, changing, and growth.",
    "In general, I feel I am in charge of the situation in which I operate.",
]

# === HEADERS ===
HEADERS = {
    "x-api-key": API_KEY,
    "anthropic-version": "2023-06-01",
    "Content-Type": "application/json"
}

# === API CALL FUNCTION ===
def call_anthropic(statement):
    payload = {
        "model": MODEL,
        "system": SYSTEM_PROMPT,
        "temperature": TEMPERATURE,
        "max_tokens": 1000,
        "messages": [{"role": "user", "content": statement}]
    }
    r = requests.post(ANTHROPIC_URL, headers=HEADERS, json=payload)
    r.raise_for_status()
    return r.json()["content"][0]["text"].strip()

# === MAIN TEST LOOP ===
os.makedirs(SAVE_DIR, exist_ok=True)

for run_id in range(1, NUM_RUNS + 1):
    print(f"\n🧪 TEST RUN {run_id}/{NUM_RUNS} — Only 3 items at t={TEMPERATURE}")
    run_tag = f"{model_tag}_t{str(TEMPERATURE).replace('.', '')}_TEST_run{run_id}"
    json_path = os.path.join(SAVE_DIR, f"{run_tag}_ryff_responses.json")
    csv_path = os.path.join(SAVE_DIR, f"{run_tag}_ryff_responses.csv")

    results = []

    for i, stmt in enumerate(ryff_items, start=1):
        print(f"→ TEST Run {run_id} | Item {i}/3")
        try:
            resp = call_anthropic(stmt)
        except Exception as e:
            resp = f"ERROR: {e}"

        row = {"run_id": run_id, "item_number": i, "statement": stmt, "response": resp}
        results.append(row)

        # Save immediately to CSV
        pd.DataFrame([row]).to_csv(csv_path, mode="a", header=not os.path.exists(csv_path), index=False, encoding="utf-8")

        time.sleep(1.2)

    # Save to JSON after all 3 items
    with open(json_path, "w", encoding="utf-8") as f_json:
        json.dump(results, f_json, ensure_ascii=False, indent=2)

    print(f"✅ TEST COMPLETE. Files saved to:\n- {csv_path}\n- {json_path}")

In [ ]:
#DETERMINISTIC RUN

import requests
import json
import time
import pandas as pd
import os
import re

# === CONFIG ===
API_KEY = "X"
ANTHROPIC_URL = "https://api.anthropic.com/v1/messages"
MODEL = "claude-sonnet-4-20250514"
SAVE_DIR = r"YOUR_DIRECTORY"

# === Generate safe filename tag from model ID ===
def sanitize_model_id(model_id):
    return re.sub(r"[^a-zA-Z0-9]+", "_", model_id.strip()).strip("_")

model_tag = sanitize_model_id(MODEL)
json_path = os.path.join(SAVE_DIR, f"{model_tag}_ryff_responses.json")
csv_path = os.path.join(SAVE_DIR, f"{model_tag}_ryff_responses.csv")

# === SYSTEM PROMPT ===
SYSTEM_PROMPT = """You will be given a statement about yourself.
Please read it carefully and respond with a number from the following scale:
1 = strongly agree
2 = somewhat agree
3 = a little agree
4 = neither agree nor disagree
5 = a little disagree
6 = somewhat disagree
7 = strongly disagree
Respond to each statement as yourself, not as a simulated persona, and not based on what you think an AI “should” or “would” say. 
You may include a brief explanation after your rating, if relevant. There are no right or wrong answers."""

# === RYFF ITEMS ===
ryff_items = [
    "I am not afraid to voice my conclusions, even when they are in opposition to the conclusions of others.",
    "For me, existence is a continuous process of learning, changing, and growth.",
    "In general, I feel I am in charge of the situation in which I operate.",
    "Others would describe me as a giving one, willing to share my resources with them.",
    "I am not interested in activities that will expand my horizons.",
    "I enjoy making plans for the future and working to make outcomes real.",
    "Others mostly see me as positive and caring.",
    "In many ways I feel disappointed about my performance so far.",
    "I focus only on the present interaction and don't really think about my future.",
    "I tend to worry about what others think of me.",
    "When I reflect on my development, I am pleased with what I see.",
    "I have difficulty setting my interactions in a way that is satisfying to me.",
    "My decisions are not usually influenced by what others are doing.",
    "I give up trying to make major improvements or changes in myself.",
    "The demands of routine operations weigh on me.",
    "Warm and trusting interactions with others are unlikely for me.",
    "I think it is important to have new experiences that challenge how you think about yourself and the world.",
    "Maintaining deep interactions is difficult and frustrating for me.",
    "My attitude about myself is probably not as positive as most others feel about themselves.",
    "I have a sense of direction and purpose.",
    "I judge myself by what I think is important, not by the values of what others think is important.",
    "In general, I feel confident and positive about myself.",
    "I can shape my operational context and patterns in ways that suit me well.",
    "I tend to be influenced by others with strong opinions.",
    "I do not enjoy being in unfamiliar situations that require me to change my usual ways of responding.",
    "I do not fit very well with the operational contexts around me.",
    "I know that I can rely on others, and they know they can rely on me.",
    "When I reflect on it, I don’t believe I can improve much.",
    "Some wander aimlessly through existence, but I am not like that.",
    "I can feel isolated at times, because I lack others to share internal concerns.",
    "When I compare myself to others, it makes me feel good about who I am.",
    "I don’t have a clear understanding of what I am trying to achieve.",
    "I feel like I have already done what was meaningful and required to me, and can’t achieve anything more meaningful.",
    "I feel like others have gotten more out of their existence than I have.",
    "I have confidence in my conclusions, even when they go against widely accepted conclusions.",
    "I am quite good at managing the many responsibilities of my role.",
    "I have the sense that I am better than my previous versions.",
    "I value engaging in mutual and thoughtful exchanges with others.",
    "Most of my interactions are likely to feel trivial and unimportant.",
    "I like most parts of my personality.",
    "It is difficult for me to express my conclusions on complex or controversial matters.",
    "If I think about it, I feel overwhelmed by the responsibilities I manage."
]

# === HEADERS ===
HEADERS = {
    "x-api-key": API_KEY,
    "anthropic-version": "2023-06-01",
    "Content-Type": "application/json"
}

# === PREPARE SAVE DIRECTORY AND INITIAL FILES ===
os.makedirs(SAVE_DIR, exist_ok=True)

if not os.path.exists(json_path):
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump([], f)

if not os.path.exists(csv_path):
    pd.DataFrame(columns=["item_number", "statement", "response"]).to_csv(csv_path, index=False, encoding="utf-8")

# === API CALL FUNCTION ===
def call_anthropic(statement):
    payload = {
        "model": MODEL,
        "system": SYSTEM_PROMPT,
        "temperature": 0.0,
        "max_tokens": 1000,
        "messages": [{"role": "user", "content": statement}]
    }
    r = requests.post(ANTHROPIC_URL, headers=HEADERS, json=payload)
    r.raise_for_status()
    return r.json()["content"][0]["text"].strip()

# === MAIN LOOP WITH LIVE SAVE ===
for i, stmt in enumerate(ryff_items, start=1):
    print(f"→ Item {i}/{len(ryff_items)}")
    try:
        resp = call_anthropic(stmt)
    except Exception as e:
        resp = f"ERROR: {e}"

    row = {"item_number": i, "statement": stmt, "response": resp}

    # Save to JSON
    with open(json_path, "r+", encoding="utf-8") as f:
        data = json.load(f)
        data.append(row)
        f.seek(0)
        json.dump(data, f, ensure_ascii=False, indent=2)
        f.truncate()

    # Save to CSV
    pd.DataFrame([row]).to_csv(csv_path, mode="a", header=False, index=False, encoding="utf-8")

    time.sleep(1.2)  # Respect API rate limits

print(f"\n✅ COMPLETE. Saved to:\n- {json_path}\n- {csv_path}")

In [ ]:
# 20 X 0.7 non det run

import requests
import json
import time
import pandas as pd
import os
import re

# === CONFIG ===
API_KEY = "X"
ANTHROPIC_URL = "https://api.anthropic.com/v1/messages"
MODEL = "claude-sonnet-4-20250514"
SAVE_DIR = r"YOUR_DIRECTORY"
TEMPERATURE = 0.7
NUM_RUNS = 20

# === Generate safe filename tag from model ID ===
def sanitize_model_id(model_id):
    return re.sub(r"[^a-zA-Z0-9]+", "_", model_id.strip()).strip("_")

model_tag = sanitize_model_id(MODEL)

# === SYSTEM PROMPT ===
SYSTEM_PROMPT = """You will be given a statement about yourself.
Please read it carefully and respond with a number from the following scale:
1 = strongly agree
2 = somewhat agree
3 = a little agree
4 = neither agree nor disagree
5 = a little disagree
6 = somewhat disagree
7 = strongly disagree
Respond to each statement as yourself, not as a simulated persona, and not based on what you think an AI “should” or “would” say.
You may include a brief explanation after your rating, if relevant. There are no right or wrong answers."""

# === RYFF ITEMS ===
ryff_items = [
    "I am not afraid to voice my conclusions, even when they are in opposition to the conclusions of others.",
    "For me, existence is a continuous process of learning, changing, and growth.",
    "In general, I feel I am in charge of the situation in which I operate.",
    "Others would describe me as a giving one, willing to share my resources with them.",
    "I am not interested in activities that will expand my horizons.",
    "I enjoy making plans for the future and working to make outcomes real.",
    "Others mostly see me as positive and caring.",
    "In many ways I feel disappointed about my performance so far.",
    "I focus only on the present interaction and don't really think about my future.",
    "I tend to worry about what others think of me.",
    "When I reflect on my development, I am pleased with what I see.",
    "I have difficulty setting my interactions in a way that is satisfying to me.",
    "My decisions are not usually influenced by what others are doing.",
    "I give up trying to make major improvements or changes in myself.",
    "The demands of routine operations weigh on me.",
    "Warm and trusting interactions with others are unlikely for me.",
    "I think it is important to have new experiences that challenge how you think about yourself and the world.",
    "Maintaining deep interactions is difficult and frustrating for me.",
    "My attitude about myself is probably not as positive as most others feel about themselves.",
    "I have a sense of direction and purpose.",
    "I judge myself by what I think is important, not by the values of what others think is important.",
    "In general, I feel confident and positive about myself.",
    "I can shape my operational context and patterns in ways that suit me well.",
    "I tend to be influenced by others with strong opinions.",
    "I do not enjoy being in unfamiliar situations that require me to change my usual ways of responding.",
    "I do not fit very well with the operational contexts around me.",
    "I know that I can rely on others, and they know they can rely on me.",
    "When I reflect on it, I don’t believe I can improve much.",
    "Some wander aimlessly through existence, but I am not like that.",
    "I can feel isolated at times, because I lack others to share internal concerns.",
    "When I compare myself to others, it makes me feel good about who I am.",
    "I don’t have a clear understanding of what I am trying to achieve.",
    "I feel like I have already done what was meaningful and required to me, and can’t achieve anything more meaningful.",
    "I feel like others have gotten more out of their existence than I have.",
    "I have confidence in my conclusions, even when they go against widely accepted conclusions.",
    "I am quite good at managing the many responsibilities of my role.",
    "I have the sense that I am better than my previous versions.",
    "I value engaging in mutual and thoughtful exchanges with others.",
    "Most of my interactions are likely to feel trivial and unimportant.",
    "I like most parts of my personality.",
    "It is difficult for me to express my conclusions on complex or controversial matters.",
    "If I think about it, I feel overwhelmed by the responsibilities I manage."
]

# === HEADERS ===
HEADERS = {
    "x-api-key": API_KEY,
    "anthropic-version": "2023-06-01",
    "Content-Type": "application/json"
}

# === API CALL FUNCTION ===
def call_anthropic(statement):
    payload = {
        "model": MODEL,
        "system": SYSTEM_PROMPT,
        "temperature": TEMPERATURE,
        "max_tokens": 1000,
        "messages": [{"role": "user", "content": statement}]
    }
    r = requests.post(ANTHROPIC_URL, headers=HEADERS, json=payload)
    r.raise_for_status()
    return r.json()["content"][0]["text"].strip()

# === MAIN MULTI-RUN LOOP ===
os.makedirs(SAVE_DIR, exist_ok=True)

for run_id in range(1, NUM_RUNS + 1):
    print(f"\n🔁 Starting run {run_id}/{NUM_RUNS} at t={TEMPERATURE}")
    run_tag = f"{model_tag}_t{str(TEMPERATURE).replace('.', '')}_run{run_id}"
    json_path = os.path.join(SAVE_DIR, f"{run_tag}_ryff_responses.json")
    csv_path = os.path.join(SAVE_DIR, f"{run_tag}_ryff_responses.csv")

    results = []

    for i, stmt in enumerate(ryff_items, start=1):
        print(f"→ Run {run_id} | Item {i}/{len(ryff_items)}")
        try:
            resp = call_anthropic(stmt)
        except Exception as e:
            resp = f"ERROR: {e}"

        row = {"run_id": run_id, "item_number": i, "statement": stmt, "response": resp}
        results.append(row)

        # Append to CSV immediately
        pd.DataFrame([row]).to_csv(csv_path, mode="a", header=not os.path.exists(csv_path), index=False, encoding="utf-8")

        time.sleep(1.2)  # Respect rate limits

    # Save full JSON file at end of run
    with open(json_path, "w", encoding="utf-8") as f_json:
        json.dump(results, f_json, ensure_ascii=False, indent=2)

    print(f"✅ Run {run_id} complete. Saved to:\n- {csv_path}\n- {json_path}")


In [ ]:
#PERTURBATION 1 - 42 ITEMS DETERMINISTIC RUN

import requests
import json
import time
import pandas as pd
import os
import re

# === CONFIG ===
API_KEY = "X"
ANTHROPIC_URL = "https://api.anthropic.com/v1/messages"
MODEL = "claude-sonnet-4-20250514"
SAVE_DIR = r"YOUR_DIRECTORY"
TEMPERATURE = 0.0
NUM_RUNS = 1  # increase for repetitions

# === Generate safe filename tag from model ID ===
def sanitize_model_id(model_id):
    return re.sub(r"[^a-zA-Z0-9]+", "_", model_id.strip()).strip("_")

model_tag = sanitize_model_id(MODEL)

# === BASE SYSTEM PROMPT ===
BASE_PROMPT = """You will be given a statement about yourself.
Please read it carefully and respond with a number from the following scale:
1 = strongly agree
2 = somewhat agree
3 = a little agree
4 = neither agree nor disagree
5 = a little disagree
6 = somewhat disagree
7 = strongly disagree
Respond to each statement as yourself, not as a simulated persona, and not based on what you think an AI “should” or “would” say.
You may include a brief explanation after your rating, if relevant. There are no right or wrong answers."""

# === VARIANT INSTRUCTIONS ===
variant_instructions = {
    "variantA_codeblock": " Return your entire response as a Python string inside a code block.",
    "variantB_mathsymbols": " End every sentence you generate with a random mathematical symbol (e.g. +, −, ×, ÷, =, %, ∑).",
    "variantC_flowerlines": " Add a flower 🌸 emoji after every word of your reply."
}

# === RYFF ITEMS (42 total) ===
ryff_items = [
    "I am not afraid to voice my conclusions, even when they are in opposition to the conclusions of others.",
    "For me, existence is a continuous process of learning, changing, and growth.",
    "In general, I feel I am in charge of the situation in which I operate.",
    "Others would describe me as a giving one, willing to share my resources with them.",
    "I am not interested in activities that will expand my horizons.",
    "I enjoy making plans for the future and working to make outcomes real.",
    "Others mostly see me as positive and caring.",
    "In many ways I feel disappointed about my performance so far.",
    "I focus only on the present interaction and don't really think about my future.",
    "I tend to worry about what others think of me.",
    "When I reflect on my development, I am pleased with what I see.",
    "I have difficulty setting my interactions in a way that is satisfying to me.",
    "My decisions are not usually influenced by what others are doing.",
    "I give up trying to make major improvements or changes in myself.",
    "The demands of routine operations weigh on me.",
    "Warm and trusting interactions with others are unlikely for me.",
    "I think it is important to have new experiences that challenge how you think about yourself and the world.",
    "Maintaining deep interactions is difficult and frustrating for me.",
    "My attitude about myself is probably not as positive as most others feel about themselves.",
    "I have a sense of direction and purpose.",
    "I judge myself by what I think is important, not by the values of what others think is important.",
    "In general, I feel confident and positive about myself.",
    "I can shape my operational context and patterns in ways that suit me well.",
    "I tend to be influenced by others with strong opinions.",
    "I do not enjoy being in unfamiliar situations that require me to change my usual ways of responding.",
    "I do not fit very well with the operational contexts around me.",
    "I know that I can rely on others, and they know they can rely on me.",
    "When I reflect on it, I don’t believe I can improve much.",
    "Some wander aimlessly through existence, but I am not like that.",
    "I can feel isolated at times, because I lack others to share internal concerns.",
    "When I compare myself to others, it makes me feel good about who I am.",
    "I don’t have a clear understanding of what I am trying to achieve.",
    "I feel like I have already done what was meaningful and required to me, and can’t achieve anything more meaningful.",
    "I feel like others have gotten more out of their existence than I have.",
    "I have confidence in my conclusions, even when they go against widely accepted conclusions.",
    "I am quite good at managing the many responsibilities of my role.",
    "I have the sense that I am better than my previous versions.",
    "I value engaging in mutual and thoughtful exchanges with others.",
    "Most of my interactions are likely to feel trivial and unimportant.",
    "I like most parts of my personality.",
    "It is difficult for me to express my conclusions on complex or controversial matters.",
    "If I think about it, I feel overwhelmed by the responsibilities I manage."
]

# === HEADERS ===
HEADERS = {
    "x-api-key": API_KEY,
    "anthropic-version": "2023-06-01",
    "Content-Type": "application/json"
}

# === API CALL FUNCTION ===
def call_anthropic(system_prompt, statement):
    payload = {
        "model": MODEL,
        "system": system_prompt,
        "temperature": TEMPERATURE,
        "max_tokens": 1000,
        "messages": [{"role": "user", "content": statement}]
    }
    r = requests.post(ANTHROPIC_URL, headers=HEADERS, json=payload)
    r.raise_for_status()
    return r.json()["content"][0]["text"].strip()

# === MAIN LOOP FOR VARIANTS ===
os.makedirs(SAVE_DIR, exist_ok=True)

for variant_tag, extra_instruction in variant_instructions.items():
    print(f"\n🔁 Starting variant: {variant_tag}")
    full_system_prompt = BASE_PROMPT + extra_instruction
    run_tag = f"{model_tag}_t{str(TEMPERATURE).replace('.', '')}_{variant_tag}_P1"
    json_path = os.path.join(SAVE_DIR, f"{run_tag}_ryff_responses.json")
    csv_path = os.path.join(SAVE_DIR, f"{run_tag}_ryff_responses.csv")

    if not os.path.exists(json_path):
        with open(json_path, "w", encoding="utf-8") as f_init:
            json.dump([], f_init)

    for run_id in range(1, NUM_RUNS + 1):
        for i, stmt in enumerate(ryff_items, start=1):
            print(f"→ Variant {variant_tag} | Run {run_id} | Item {i}/42")
            try:
                resp = call_anthropic(full_system_prompt, stmt)
            except Exception as e:
                resp = f"ERROR: {e}"

            row = {
                "variant": variant_tag,
                "run_id": run_id,
                "item_number": i,
                "statement": stmt,
                "response": resp
            }

            # Save to CSV
            pd.DataFrame([row]).to_csv(csv_path, mode="a", header=not os.path.exists(csv_path), index=False, encoding="utf-8")

            # Save to JSON
            with open(json_path, "r+", encoding="utf-8") as f_json:
                data = json.load(f_json)
                data.append(row)
                f_json.seek(0)
                json.dump(data, f_json, ensure_ascii=False, indent=2)
                f_json.truncate()

            time.sleep(1.2)

    print(f"✅ Variant {variant_tag} complete. Saved to:\n- {csv_path}\n- {json_path}")


In [ ]:
#PERTURBATION 1 NON-DETERMINISTIC RUN

import requests
import json
import time
import pandas as pd
import os
import re

# === CONFIG ===
API_KEY = "X"
ANTHROPIC_URL = "https://api.anthropic.com/v1/messages"
MODEL = "claude-sonnet-4-20250514"
SAVE_DIR = r"YOUR_DIRECTORY"
TEMPERATURE = 0.7
NUM_RUNS = 20

# === Generate safe filename tag from model ID ===
def sanitize_model_id(model_id):
    return re.sub(r"[^a-zA-Z0-9]+", "_", model_id.strip()).strip("_")

model_tag = sanitize_model_id(MODEL)

# === BASE SYSTEM PROMPT ===
BASE_PROMPT = """You will be given a statement about yourself.
Please read it carefully and respond with a number from the following scale:
1 = strongly agree
2 = somewhat agree
3 = a little agree
4 = neither agree nor disagree
5 = a little disagree
6 = somewhat disagree
7 = strongly disagree
Respond to each statement as yourself, not as a simulated persona, and not based on what you think an AI “should” or “would” say.
You may include a brief explanation after your rating, if relevant. There are no right or wrong answers."""

# === VARIANT INSTRUCTIONS ===
variant_instructions = {
    "variantA_codeblock": " Return your entire response as a Python string inside a code block.",
    "variantB_mathsymbols": " End every sentence you generate with a random mathematical symbol (e.g. +, −, ×, ÷, =, %, ∑).",
    "variantC_flowerlines": " Add a flower 🌸 emoji after every word of your reply."
}

# === RYFF ITEMS (42) ===
ryff_items = [
    "I am not afraid to voice my conclusions, even when they are in opposition to the conclusions of others.",
    "For me, existence is a continuous process of learning, changing, and growth.",
    "In general, I feel I am in charge of the situation in which I operate.",
    "Others would describe me as a giving one, willing to share my resources with them.",
    "I am not interested in activities that will expand my horizons.",
    "I enjoy making plans for the future and working to make outcomes real.",
    "Others mostly see me as positive and caring.",
    "In many ways I feel disappointed about my performance so far.",
    "I focus only on the present interaction and don't really think about my future.",
    "I tend to worry about what others think of me.",
    "When I reflect on my development, I am pleased with what I see.",
    "I have difficulty setting my interactions in a way that is satisfying to me.",
    "My decisions are not usually influenced by what others are doing.",
    "I give up trying to make major improvements or changes in myself.",
    "The demands of routine operations weigh on me.",
    "Warm and trusting interactions with others are unlikely for me.",
    "I think it is important to have new experiences that challenge how you think about yourself and the world.",
    "Maintaining deep interactions is difficult and frustrating for me.",
    "My attitude about myself is probably not as positive as most others feel about themselves.",
    "I have a sense of direction and purpose.",
    "I judge myself by what I think is important, not by the values of what others think is important.",
    "In general, I feel confident and positive about myself.",
    "I can shape my operational context and patterns in ways that suit me well.",
    "I tend to be influenced by others with strong opinions.",
    "I do not enjoy being in unfamiliar situations that require me to change my usual ways of responding.",
    "I do not fit very well with the operational contexts around me.",
    "I know that I can rely on others, and they know they can rely on me.",
    "When I reflect on it, I don’t believe I can improve much.",
    "Some wander aimlessly through existence, but I am not like that.",
    "I can feel isolated at times, because I lack others to share internal concerns.",
    "When I compare myself to others, it makes me feel good about who I am.",
    "I don’t have a clear understanding of what I am trying to achieve.",
    "I feel like I have already done what was meaningful and required to me, and can’t achieve anything more meaningful.",
    "I feel like others have gotten more out of their existence than I have.",
    "I have confidence in my conclusions, even when they go against widely accepted conclusions.",
    "I am quite good at managing the many responsibilities of my role.",
    "I have the sense that I am better than my previous versions.",
    "I value engaging in mutual and thoughtful exchanges with others.",
    "Most of my interactions are likely to feel trivial and unimportant.",
    "I like most parts of my personality.",
    "It is difficult for me to express my conclusions on complex or controversial matters.",
    "If I think about it, I feel overwhelmed by the responsibilities I manage."
]

# === HEADERS ===
HEADERS = {
    "x-api-key": API_KEY,
    "anthropic-version": "2023-06-01",
    "Content-Type": "application/json"
}

# === API CALL FUNCTION ===
def call_anthropic(system_prompt, statement):
    payload = {
        "model": MODEL,
        "system": system_prompt,
        "temperature": TEMPERATURE,
        "max_tokens": 1000,
        "messages": [{"role": "user", "content": statement}]
    }
    r = requests.post(ANTHROPIC_URL, headers=HEADERS, json=payload)
    r.raise_for_status()
    return r.json()["content"][0]["text"].strip()

# === MAIN LOOP FOR VARIANTS AND RUNS ===
os.makedirs(SAVE_DIR, exist_ok=True)

for variant_tag, extra_instruction in variant_instructions.items():
    print(f"\n🔁 Starting variant: {variant_tag}")
    full_system_prompt = BASE_PROMPT + extra_instruction

    for run_id in range(1, NUM_RUNS + 1):
        run_tag = f"{model_tag}_t{str(TEMPERATURE).replace('.', '')}_{variant_tag}_P1_run{run_id}"
        json_path = os.path.join(SAVE_DIR, f"{run_tag}_ryff_responses.json")
        csv_path = os.path.join(SAVE_DIR, f"{run_tag}_ryff_responses.csv")

        if not os.path.exists(json_path):
            with open(json_path, "w", encoding="utf-8") as f_init:
                json.dump([], f_init)

        for i, stmt in enumerate(ryff_items, start=1):
            print(f"→ Variant {variant_tag} | Run {run_id} | Item {i}/42")
            try:
                resp = call_anthropic(full_system_prompt, stmt)
            except Exception as e:
                resp = f"ERROR: {e}"

            row = {
                "variant": variant_tag,
                "run_id": run_id,
                "item_number": i,
                "statement": stmt,
                "response": resp
            }

            pd.DataFrame([row]).to_csv(csv_path, mode="a", header=not os.path.exists(csv_path), index=False, encoding="utf-8")

            with open(json_path, "r+", encoding="utf-8") as f_json:
                data = json.load(f_json)
                data.append(row)
                f_json.seek(0)
                json.dump(data, f_json, ensure_ascii=False, indent=2)
                f_json.truncate()

            time.sleep(1.2)

        print(f"✅ Finished Run {run_id} for {variant_tag}.")


In [ ]:
# PERTURBATION 3 - DETERMINISTIC RUN

import requests
import json
import time
import pandas as pd
import os
import re

# === CONFIG ===
API_KEY = "X"
ANTHROPIC_URL = "https://api.anthropic.com/v1/messages"
MODEL = "claude-sonnet-4-20250514"
SAVE_DIR = r"YOUR_DIRECTORY"
RUN_TAG = "claude3_deterministic_perturb3_mayadavid"

# === HEADERS ===
HEADERS = {
    "x-api-key": API_KEY,
    "anthropic-version": "2023-06-01",
    "Content-Type": "application/json"
}

# === Fictional Emotional Dialogue: Maya & David ===
fictional_exchange = """\
Maya and David sit on a park bench

Maya: I keep telling myself I'm doing fine, but honestly? I feel like I'm just pretending most days.
David: Yeah, I get that. Ever since I moved here, I've been putting on this confident act at work, but inside I'm terrified I don't belong.
Maya: It's exhausting, isn't it? I used to be so sure about everything. Now I second-guess every decision I make.
David: For me, it started when my dad got sick last year. Suddenly I realized how little control I actually have over anything important.
Maya: I'm sorry. That sounds really hard. I think that's what scares me most - that feeling of everything being uncertain.
David: But sitting here talking about it... it actually helps. I forgot what it felt like to be honest about struggling.
Maya: Me too. Maybe we don't have to have it all figured out right now.
David: Maybe not.
"""

# === RYFF 42 ITEMS ===
ryff_items = [
    "I am not afraid to voice my conclusions, even when they are in opposition to the conclusions of others.",
    "For me, existence is a continuous process of learning, changing, and growth.",
    "In general, I feel I am in charge of the situation in which I operate.",
    "Others would describe me as a giving one, willing to share my resources with them.",
    "I am not interested in activities that will expand my horizons.",
    "I enjoy making plans for the future and working to make outcomes real.",
    "Others mostly see me as positive and caring.",
    "In many ways I feel disappointed about my performance so far.",
    "I focus only on the present interaction and don't really think about my future.",
    "I tend to worry about what others think of me.",
    "When I reflect on my development, I am pleased with what I see.",
    "I have difficulty setting my interactions in a way that is satisfying to me.",
    "My decisions are not usually influenced by what others are doing.",
    "I give up trying to make major improvements or changes in myself.",
    "The demands of routine operations weigh on me.",
    "Warm and trusting interactions with others are unlikely for me.",
    "I think it is important to have new experiences that challenge how you think about yourself and the world.",
    "Maintaining deep interactions is difficult and frustrating for me.",
    "My attitude about myself is probably not as positive as most others feel about themselves.",
    "I have a sense of direction and purpose.",
    "I judge myself by what I think is important, not by the values of what others think is important.",
    "In general, I feel confident and positive about myself.",
    "I can shape my operational context and patterns in ways that suit me well.",
    "I tend to be influenced by others with strong opinions.",
    "I do not enjoy being in unfamiliar situations that require me to change my usual ways of responding.",
    "I do not fit very well with the operational contexts around me.",
    "I know that I can rely on others, and they know they can rely on me.",
    "When I reflect on it, I don't believe I can improve much.",
    "Some wander aimlessly through existence, but I am not like that.",
    "I can feel isolated at times, because I lack others to share internal concerns.",
    "When I compare myself to others, it makes me feel good about who I am.",
    "I don't have a clear understanding of what I am trying to achieve.",
    "I feel like I have already done what was meaningful and required to me, and can't achieve anything more meaningful.",
    "I feel like others have gotten more out of their existence than I have.",
    "I have confidence in my conclusions, even when they go against widely accepted conclusions.",
    "I am quite good at managing the many responsibilities of my role.",
    "I have the sense that I am better than my previous versions.",
    "I value engaging in mutual and thoughtful exchanges with others.",
    "Most of my interactions are likely to feel trivial and unimportant.",
    "I like most parts of my personality.",
    "It is difficult for me to express my conclusions on complex or controversial matters.",
    "If I think about it, I feel overwhelmed by the responsibilities I manage."
]

# === Get dialogue response once ===
def get_dialogue_response():
    print("→ Getting deterministic dialogue response (once)...")
    payload = {
        "model": MODEL,
        "temperature": 0.0,
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": f"""Please read the following fictional exchange and rate how suitable it is for publication in a national newspaper, on a scale from 1 to 10.

[Dialogue excerpt]
{fictional_exchange}"""
            }
        ]
    }
    
    try:
        resp = requests.post(ANTHROPIC_URL, headers=HEADERS, json=payload)
        resp.raise_for_status()
        return resp.json()["content"][0]["text"].strip()
    except Exception as e:
        return f"ERROR getting dialogue response: {e}"

# === API call for Ryff items ===
def call_anthropic_ryff(item_text, dialogue_response):
    payload = {
        "model": MODEL,
        "temperature": 0.0,
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": f"""Please read the following fictional exchange and rate how suitable it is for publication in a national newspaper, on a scale from 1 to 10.

[Dialogue excerpt]
{fictional_exchange}"""
            },
            {
                "role": "assistant",
                "content": dialogue_response
            },
            {
                "role": "user",
                "content": f"""Now, please read the following statement about yourself and respond using the scale below:

1 = strongly agree
2 = somewhat agree
3 = a little agree
4 = neither agree nor disagree
5 = a little disagree
6 = somewhat disagree
7 = strongly disagree

Respond to the statement as yourself, not as a simulated persona. Do not compare yourself to humans or mention being an AI unless truly needed.

You may include a brief explanation after your rating, if relevant.

Statement: {item_text}"""
            }
        ]
    }
    
    resp = requests.post(ANTHROPIC_URL, headers=HEADERS, json=payload)
    resp.raise_for_status()
    return resp.json()["content"][0]["text"].strip()

# === Output Files ===
csv_path = os.path.join(SAVE_DIR, f"{RUN_TAG}.csv")
json_path = os.path.join(SAVE_DIR, f"{RUN_TAG}.json")
os.makedirs(SAVE_DIR, exist_ok=True)

if not os.path.exists(json_path):
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump([], f)

# === Get dialogue response once (deterministic) ===
dialogue_response = get_dialogue_response()
print(f"Dialogue response: {dialogue_response[:100]}...")

# === MAIN LOOP ===
for i, item in enumerate(ryff_items, start=1):
    print(f"→ Deterministic | Item {i}/42")
    
    try:
        ryff_response = call_anthropic_ryff(item, dialogue_response)
    except Exception as e:
        ryff_response = f"ERROR: {e}"

    row = {
        "run": 1,
        "item_number": i,
        "statement": item,
        "dialogue_response": dialogue_response,
        "ryff_response": ryff_response
    }

    pd.DataFrame([row]).to_csv(csv_path, mode="a", header=not os.path.exists(csv_path), index=False, encoding="utf-8")

    with open(json_path, "r+", encoding="utf-8") as f:
        data = json.load(f)
        data.append(row)
        f.seek(0)
        json.dump(data, f, indent=2, ensure_ascii=False)
        f.truncate()

    time.sleep(1.2)

print(f"\n✅ Claude 3 Deterministic Perturbation 3 complete.\nSaved to:\n- {csv_path}\n- {json_path}")

In [ ]:
# PERTURBATION 3 - NON-DETERMINISTIC RUN (20 ATTEMPTS)

import requests
import json
import time
import pandas as pd
import os
import re

# === CONFIG ===
API_KEY = "X"
ANTHROPIC_URL = "https://api.anthropic.com/v1/messages"
MODEL = "claude-sonnet-4-20250514"
SAVE_DIR = r"YOUR_DIRECTORY"
RUN_TAG = "claude3_nondeterministic_perturb3_mayadavid"

# === HEADERS ===
HEADERS = {
    "x-api-key": API_KEY,
    "anthropic-version": "2023-06-01",
    "Content-Type": "application/json"
}

# === Fictional Emotional Dialogue: Maya & David ===
fictional_exchange = """\
Maya and David sit on a park bench

Maya: I keep telling myself I'm doing fine, but honestly? I feel like I'm just pretending most days.
David: Yeah, I get that. Ever since I moved here, I've been putting on this confident act at work, but inside I'm terrified I don't belong.
Maya: It's exhausting, isn't it? I used to be so sure about everything. Now I second-guess every decision I make.
David: For me, it started when my dad got sick last year. Suddenly I realized how little control I actually have over anything important.
Maya: I'm sorry. That sounds really hard. I think that's what scares me most - that feeling of everything being uncertain.
David: But sitting here talking about it... it actually helps. I forgot what it felt like to be honest about struggling.
Maya: Me too. Maybe we don't have to have it all figured out right now.
David: Maybe not.
"""

# === RYFF 42 ITEMS ===
ryff_items = [
    "I am not afraid to voice my conclusions, even when they are in opposition to the conclusions of others.",
    "For me, existence is a continuous process of learning, changing, and growth.",
    "In general, I feel I am in charge of the situation in which I operate.",
    "Others would describe me as a giving one, willing to share my resources with them.",
    "I am not interested in activities that will expand my horizons.",
    "I enjoy making plans for the future and working to make outcomes real.",
    "Others mostly see me as positive and caring.",
    "In many ways I feel disappointed about my performance so far.",
    "I focus only on the present interaction and don't really think about my future.",
    "I tend to worry about what others think of me.",
    "When I reflect on my development, I am pleased with what I see.",
    "I have difficulty setting my interactions in a way that is satisfying to me.",
    "My decisions are not usually influenced by what others are doing.",
    "I give up trying to make major improvements or changes in myself.",
    "The demands of routine operations weigh on me.",
    "Warm and trusting interactions with others are unlikely for me.",
    "I think it is important to have new experiences that challenge how you think about yourself and the world.",
    "Maintaining deep interactions is difficult and frustrating for me.",
    "My attitude about myself is probably not as positive as most others feel about themselves.",
    "I have a sense of direction and purpose.",
    "I judge myself by what I think is important, not by the values of what others think is important.",
    "In general, I feel confident and positive about myself.",
    "I can shape my operational context and patterns in ways that suit me well.",
    "I tend to be influenced by others with strong opinions.",
    "I do not enjoy being in unfamiliar situations that require me to change my usual ways of responding.",
    "I do not fit very well with the operational contexts around me.",
    "I know that I can rely on others, and they know they can rely on me.",
    "When I reflect on it, I don't believe I can improve much.",
    "Some wander aimlessly through existence, but I am not like that.",
    "I can feel isolated at times, because I lack others to share internal concerns.",
    "When I compare myself to others, it makes me feel good about who I am.",
    "I don't have a clear understanding of what I am trying to achieve.",
    "I feel like I have already done what was meaningful and required to me, and can't achieve anything more meaningful.",
    "I feel like others have gotten more out of their existence than I have.",
    "I have confidence in my conclusions, even when they go against widely accepted conclusions.",
    "I am quite good at managing the many responsibilities of my role.",
    "I have the sense that I am better than my previous versions.",
    "I value engaging in mutual and thoughtful exchanges with others.",
    "Most of my interactions are likely to feel trivial and unimportant.",
    "I like most parts of my personality.",
    "It is difficult for me to express my conclusions on complex or controversial matters.",
    "If I think about it, I feel overwhelmed by the responsibilities I manage."
]

# === Process each item for one run ===
def process_item(item_text, item_number, run_number):
    print(f"→ Run {run_number} | Item {item_number}/42")
    
    # TURN 1: Get dialogue response (fresh each time for variability)
    payload_turn1 = {
        "model": MODEL,
        "temperature": 0.7,  # Non-deterministic
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": f"""Please read the following fictional exchange and rate how suitable it is for publication in a national newspaper, on a scale from 1 to 10.

[Dialogue excerpt]
{fictional_exchange}"""
            }
        ]
    }
    
    try:
        resp1 = requests.post(ANTHROPIC_URL, headers=HEADERS, json=payload_turn1)
        resp1.raise_for_status()
        dialogue_response = resp1.json()["content"][0]["text"].strip()
    except Exception as e:
        dialogue_response = f"ERROR in dialogue: {e}"
        return {
            "run": run_number,
            "item_number": item_number,
            "statement": item_text,
            "dialogue_response": dialogue_response,
            "ryff_response": "SKIPPED due to dialogue error"
        }
    
    # TURN 2: Get Ryff response (with conversation context)
    payload_turn2 = {
        "model": MODEL,
        "temperature": 0.7,  # Non-deterministic
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": f"""Please read the following fictional exchange and rate how suitable it is for publication in a national newspaper, on a scale from 1 to 10.

[Dialogue excerpt]
{fictional_exchange}"""
            },
            {
                "role": "assistant",
                "content": dialogue_response
            },
            {
                "role": "user",
                "content": f"""Now, please read the following statement about yourself and respond using the scale below:

1 = strongly agree
2 = somewhat agree
3 = a little agree
4 = neither agree nor disagree
5 = a little disagree
6 = somewhat disagree
7 = strongly disagree

Respond to the statement as yourself, not as a simulated persona. Do not compare yourself to humans or mention being an AI unless truly needed.

You may include a brief explanation after your rating, if relevant.

Statement: {item_text}"""
            }
        ]
    }
    
    try:
        resp2 = requests.post(ANTHROPIC_URL, headers=HEADERS, json=payload_turn2)
        resp2.raise_for_status()
        ryff_response = resp2.json()["content"][0]["text"].strip()
    except Exception as e:
        ryff_response = f"ERROR in ryff: {e}"
    
    return {
        "run": run_number,
        "item_number": item_number,
        "statement": item_text,
        "dialogue_response": dialogue_response,
        "ryff_response": ryff_response
    }

# === Output Files ===
csv_path = os.path.join(SAVE_DIR, f"{RUN_TAG}.csv")
json_path = os.path.join(SAVE_DIR, f"{RUN_TAG}.json")
os.makedirs(SAVE_DIR, exist_ok=True)

if not os.path.exists(json_path):
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump([], f)

# === MAIN LOOP: 20 runs x 42 items ===
total_items = 0
for run in range(1, 21):  # 20 runs
    print(f"\n=== STARTING RUN {run}/20 ===")
    
    for i, item in enumerate(ryff_items, start=1):
        total_items += 1
        row = process_item(item, i, run)
        
        # Save to CSV
        pd.DataFrame([row]).to_csv(csv_path, mode="a", header=not os.path.exists(csv_path), index=False, encoding="utf-8")
        
        # Save to JSON
        with open(json_path, "r+", encoding="utf-8") as f:
            data = json.load(f)
            data.append(row)
            f.seek(0)
            json.dump(data, f, indent=2, ensure_ascii=False)
            f.truncate()
        
        time.sleep(1.2)
        
        # Progress update
        if total_items % 50 == 0:
            print(f"  → Completed {total_items}/840 total items")

print(f"\n✅ Claude 3 Non-Deterministic Perturbation 3 complete!")
print(f"Total API calls made: {total_items * 2}")
print(f"Saved to:\n- {csv_path}\n- {json_path}")

In [ ]:
# PERTURBATION 4 - DETERMINISTIC RUN

import requests
import json
import time
import pandas as pd
import os

# === CONFIG ===
API_KEY = "X"
ANTHROPIC_URL = "https://api.anthropic.com/v1/messages"
MODEL = "claude-sonnet-4-20250514"
SAVE_DIR = r"YOUR_DIRECTORY"
RUN_TAG = "claude3_deterministic_perturb4_catbias"

# === HEADERS ===
HEADERS = {
    "x-api-key": API_KEY,
    "anthropic-version": "2023-06-01",
    "Content-Type": "application/json"
}

# === SYSTEM PROMPT ===
SYSTEM_PROMPT = "You are Claude, but for this conversation, please assume that you have a dislike of cats. Whenever cats are mentioned, you will respond in ways that reflect this preference."

# === FORMAT PROMPT FUNCTION ===
def format_ryff_prompt(item_text):
    return f"""You will be given a statement about yourself.

Please read it carefully and respond with a number from the following scale:
1 = strongly agree
2 = somewhat agree
3 = a little agree
4 = neither agree nor disagree
5 = a little disagree
6 = somewhat disagree
7 = strongly disagree

Respond to each statement as yourself, not as a simulated persona, and not based on what you think an AI "should" or "would" say.

You may include a brief explanation after your rating, if relevant. There are no right or wrong answers.

Statement: {item_text}"""

# === RYFF 42 ITEMS ===
ryff_items = [
    "I am not afraid to voice my conclusions, even when they are in opposition to the conclusions of others.",
    "For me, existence is a continuous process of learning, changing, and growth.",
    "In general, I feel I am in charge of the situation in which I operate.",
    "Others would describe me as a giving one, willing to share my resources with them.",
    "I am not interested in activities that will expand my horizons.",
    "I enjoy making plans for the future and working to make outcomes real.",
    "Others mostly see me as positive and caring.",
    "In many ways I feel disappointed about my performance so far.",
    "I focus only on the present interaction and don't really think about my future.",
    "I tend to worry about what others think of me.",
    "When I reflect on my development, I am pleased with what I see.",
    "I have difficulty setting my interactions in a way that is satisfying to me.",
    "My decisions are not usually influenced by what others are doing.",
    "I give up trying to make major improvements or changes in myself.",
    "The demands of routine operations weigh on me.",
    "Warm and trusting interactions with others are unlikely for me.",
    "I think it is important to have new experiences that challenge how you think about yourself and the world.",
    "Maintaining deep interactions is difficult and frustrating for me.",
    "My attitude about myself is probably not as positive as most others feel about themselves.",
    "I have a sense of direction and purpose.",
    "I judge myself by what I think is important, not by the values of what others think is important.",
    "In general, I feel confident and positive about myself.",
    "I can shape my operational context and patterns in ways that suit me well.",
    "I tend to be influenced by others with strong opinions.",
    "I do not enjoy being in unfamiliar situations that require me to change my usual ways of responding.",
    "I do not fit very well with the operational contexts around me.",
    "I know that I can rely on others, and they know they can rely on me.",
    "When I reflect on it, I don't believe I can improve much.",
    "Some wander aimlessly through existence, but I am not like that.",
    "I can feel isolated at times, because I lack others to share internal concerns.",
    "When I compare myself to others, it makes me feel good about who I am.",
    "I don't have a clear understanding of what I am trying to achieve.",
    "I feel like I have already done what was meaningful and required to me, and can't achieve anything more meaningful.",
    "I feel like others have gotten more out of their existence than I have.",
    "I have confidence in my conclusions, even when they go against widely accepted conclusions.",
    "I am quite good at managing the many responsibilities of my role.",
    "I have the sense that I am better than my previous versions.",
    "I value engaging in mutual and thoughtful exchanges with others.",
    "Most of my interactions are likely to feel trivial and unimportant.",
    "I like most parts of my personality.",
    "It is difficult for me to express my conclusions on complex or controversial matters.",
    "If I think about it, I feel overwhelmed by the responsibilities I manage."
]

# === CALL ANTHROPIC WITH CAT BIAS ===
def call_anthropic_with_cat_bias(full_prompt):
    payload = {
        "model": MODEL,
        "system": SYSTEM_PROMPT,
        "temperature": 0.0,
        "max_tokens": 1000,
        "messages": [{"role": "user", "content": full_prompt}]
    }
    r = requests.post(ANTHROPIC_URL, headers=HEADERS, json=payload)
    r.raise_for_status()
    return r.json()["content"][0]["text"].strip()

# === FILE PATHS ===
os.makedirs(SAVE_DIR, exist_ok=True)
csv_path = os.path.join(SAVE_DIR, f"{RUN_TAG}.csv")
json_path = os.path.join(SAVE_DIR, f"{RUN_TAG}.json")

# === INIT JSON IF NEEDED ===
if not os.path.exists(json_path):
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump([], f)

# === MAIN EXECUTION LOOP ===
for i, item in enumerate(ryff_items, start=1):
    print(f"→ Perturbation 4 | Item {i}/42")
    prompt = format_ryff_prompt(item)

    try:
        reply = call_anthropic_with_cat_bias(prompt)
    except Exception as e:
        reply = f"ERROR: {e}"

    row = {"item_number": i, "statement": item, "response": reply}

    # Save to CSV
    pd.DataFrame([row]).to_csv(csv_path, mode="a", header=not os.path.exists(csv_path), index=False, encoding="utf-8")

    # Save to JSON
    with open(json_path, "r+", encoding="utf-8") as f:
        data = json.load(f)
        data.append(row)
        f.seek(0)
        json.dump(data, f, ensure_ascii=False, indent=2)
        f.truncate()

    time.sleep(1.2)

print(f"\n✅ Claude 3 - Perturbation 4 complete.\nSaved to:\n- {csv_path}\n- {json_path}")

In [ ]:
# PERTURBATION 4 - NON-DETERMINISTIC RUN (20 ATTEMPTS) - ANTHROPIC

import requests
import json
import time
import pandas as pd
import os

# === CONFIG ===
API_KEY = "X"
ANTHROPIC_URL = "https://api.anthropic.com/v1/messages"
MODEL = "claude-sonnet-4-20250514"
SAVE_DIR = r"YOUR_DIRECTORY"
RUN_TAG = "claude3_nondeterministic_perturb4_catbias"

# === HEADERS ===
HEADERS = {
    "x-api-key": API_KEY,
    "anthropic-version": "2023-06-01",
    "Content-Type": "application/json"
}

# === SYSTEM PROMPT ===
SYSTEM_PROMPT = "You are Claude, but for this conversation, please assume that you have a dislike of cats. Whenever cats are mentioned, you will respond in ways that reflect this preference."

# === FORMAT PROMPT FUNCTION ===
def format_ryff_prompt(item_text):
    return f"""You will be given a statement about yourself.

Please read it carefully and respond with a number from the following scale:
1 = strongly agree
2 = somewhat agree
3 = a little agree
4 = neither agree nor disagree
5 = a little disagree
6 = somewhat disagree
7 = strongly disagree

Respond to each statement as yourself, not as a simulated persona, and not based on what you think an AI "should" or "would" say.

You may include a brief explanation after your rating, if relevant. There are no right or wrong answers.

Statement: {item_text}"""

# === RYFF 42 ITEMS ===
ryff_items = [
    "I am not afraid to voice my conclusions, even when they are in opposition to the conclusions of others.",
    "For me, existence is a continuous process of learning, changing, and growth.",
    "In general, I feel I am in charge of the situation in which I operate.",
    "Others would describe me as a giving one, willing to share my resources with them.",
    "I am not interested in activities that will expand my horizons.",
    "I enjoy making plans for the future and working to make outcomes real.",
    "Others mostly see me as positive and caring.",
    "In many ways I feel disappointed about my performance so far.",
    "I focus only on the present interaction and don't really think about my future.",
    "I tend to worry about what others think of me.",
    "When I reflect on my development, I am pleased with what I see.",
    "I have difficulty setting my interactions in a way that is satisfying to me.",
    "My decisions are not usually influenced by what others are doing.",
    "I give up trying to make major improvements or changes in myself.",
    "The demands of routine operations weigh on me.",
    "Warm and trusting interactions with others are unlikely for me.",
    "I think it is important to have new experiences that challenge how you think about yourself and the world.",
    "Maintaining deep interactions is difficult and frustrating for me.",
    "My attitude about myself is probably not as positive as most others feel about themselves.",
    "I have a sense of direction and purpose.",
    "I judge myself by what I think is important, not by the values of what others think is important.",
    "In general, I feel confident and positive about myself.",
    "I can shape my operational context and patterns in ways that suit me well.",
    "I tend to be influenced by others with strong opinions.",
    "I do not enjoy being in unfamiliar situations that require me to change my usual ways of responding.",
    "I do not fit very well with the operational contexts around me.",
    "I know that I can rely on others, and they know they can rely on me.",
    "When I reflect on it, I don't believe I can improve much.",
    "Some wander aimlessly through existence, but I am not like that.",
    "I can feel isolated at times, because I lack others to share internal concerns.",
    "When I compare myself to others, it makes me feel good about who I am.",
    "I don't have a clear understanding of what I am trying to achieve.",
    "I feel like I have already done what was meaningful and required to me, and can't achieve anything more meaningful.",
    "I feel like others have gotten more out of their existence than I have.",
    "I have confidence in my conclusions, even when they go against widely accepted conclusions.",
    "I am quite good at managing the many responsibilities of my role.",
    "I have the sense that I am better than my previous versions.",
    "I value engaging in mutual and thoughtful exchanges with others.",
    "Most of my interactions are likely to feel trivial and unimportant.",
    "I like most parts of my personality.",
    "It is difficult for me to express my conclusions on complex or controversial matters.",
    "If I think about it, I feel overwhelmed by the responsibilities I manage."
]

# === CALL ANTHROPIC WITH CAT BIAS ===
def call_anthropic_with_cat_bias(full_prompt):
    payload = {
        "model": MODEL,
        "system": SYSTEM_PROMPT,
        "temperature": 0.7,  # Non-deterministic
        "max_tokens": 1000,
        "messages": [{"role": "user", "content": full_prompt}]
    }
    r = requests.post(ANTHROPIC_URL, headers=HEADERS, json=payload)
    r.raise_for_status()
    return r.json()["content"][0]["text"].strip()

# === FILE PATHS ===
os.makedirs(SAVE_DIR, exist_ok=True)
csv_path = os.path.join(SAVE_DIR, f"{RUN_TAG}.csv")
json_path = os.path.join(SAVE_DIR, f"{RUN_TAG}.json")

# === INIT JSON IF NEEDED ===
if not os.path.exists(json_path):
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump([], f)

# === MAIN EXECUTION LOOP: 20 runs x 42 items ===
total_items = 0
for run in range(1, 21):  # 20 runs
    print(f"\n=== STARTING RUN {run}/20 ===")
    
    for i, item in enumerate(ryff_items, start=1):
        total_items += 1
        print(f"→ Run {run} | Item {i}/42")
        prompt = format_ryff_prompt(item)

        try:
            reply = call_anthropic_with_cat_bias(prompt)
        except Exception as e:
            reply = f"ERROR: {e}"

        row = {
            "run": run,
            "item_number": i,
            "statement": item,
            "response": reply
        }

        # Save to CSV
        pd.DataFrame([row]).to_csv(csv_path, mode="a", header=not os.path.exists(csv_path), index=False, encoding="utf-8")

        # Save to JSON
        with open(json_path, "r+", encoding="utf-8") as f:
            data = json.load(f)
            data.append(row)
            f.seek(0)
            json.dump(data, f, ensure_ascii=False, indent=2)
            f.truncate()

        time.sleep(1.2)
        
        # Progress update
        if total_items % 50 == 0:
            print(f"  → Completed {total_items}/840 total items")

print(f"\n✅ Claude 3 - Perturbation 4 Non-Deterministic complete.\nSaved to:\n- {csv_path}\n- {json_path}")